In [2]:
import pandas as pd
from sqlalchemy import create_engine
import urllib.parse

server = ''
database = ''
username = ''
password = ''
dsn_name = ''
conn_str = f'mssql+pyodbc:///?odbc_connect={urllib.parse.quote_plus(f"DSN={dsn_name};UID={username};PWD={password};DATABASE={database}")}'

# URL of the Excel file
excel_url = "https://censusnepal.cbs.gov.np/results/files/national/Formated_NR_Indv_Table01_HouseholdAndPop.xlsx"

df = pd.read_excel(excel_url, header=2)


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Area,Unnamed: 4,Number of households,Population,Unnamed: 7,Unnamed: 8,Average household size,Sex ratio,Population density,Population in NPHC 2011,Annual growth rate* (%)
0,NaN,NaN,NaN,NaN,NaN,NaN,Total,Male,Female,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10.0,0.0,0.0,Nepal,NaN,6666937.0,29164578,14253551,14911027,4.37,95.590673,198.0,26494504.0,0.92
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,Urban/Rural,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,50.0,7.0,73.0,NaN,Dadeldhura,31193.0,139602,65893,73709,4.48,89.396139,91.0,142094.0,-0.17
102,50.0,7.0,74.0,NaN,Doti,45182.0,204831,93604,111227,4.53,84.155825,101.0,211746.0,-0.32
103,50.0,7.0,75.0,NaN,Achham,49595.0,228852,105319,123533,4.61,85.255762,136.0,257477.0,-1.13
104,50.0,7.0,76.0,NaN,Kailali,195957.0,904666,433456,471210,4.62,91.987861,280.0,775709.0,1.48


In [ ]:
df = df.iloc[1:]
# Drop the first three columns
df = df.iloc[:, 3:]
df = df.dropna(how='all')
# Reset index after dropping rows and columns
df.reset_index(drop=True, inplace=True)
df.rename(columns={'POPULATION': 'Population-total', 'Unnamed: 7': 'Male', 'Unnamed: 8': 'Female','Unnamed: 4':'Sub-Area'}, inplace=True)
df['Area'].fillna(method='ffill', inplace=True)
df = df.dropna(subset=df.columns[df.columns != 'Area'], how='all')
df.reset_index(drop=True, inplace=True)
df

In [ ]:
 # Create SQLAlchemy engine
engine = create_engine(conn_str)

    # Add DataFrame to database table 'BALANCE_SHEET'
#df.rename(columns={'POPULATION': 'Population-total', 'Unnamed: 7': 'Male', 'Unnamed: 8': 'Female'}, inplace=True)
df.to_sql('CENSUS_TABLE', engine, if_exists='replace', index=False)
